In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./SLAP_LOC.csv')
df.head()

,ORD_NO,SKU_CD,NUM_PCS,LOC,CART_NO,SEQ
0,ORD_0330,SKU_0242,1,WP_0001,0,1
1,ORD_0056,SKU_0274,1,WP_0120,0,2
2,ORD_0138,SKU_0287,1,WP_0155,0,3
3,ORD_0088,SKU_0046,1,WP_0156,0,4
4,ORD_0092,SKU_0022,1,WP_0156,0,5


In [2]:
ord_group = df.groupby(['ORD_NO','SKU_CD', 'LOC'])[['NUM_PCS']].count()
ord_group

NUM_PCS
ORD_NO   SKU_CD   LOC             
ORD_0001 SKU_0246 WP_0049        1
         SKU_0267 WP_0101        1
ORD_0002 SKU_0005 WP_0166        1
         SKU_0047 WP_0137        1
         SKU_0063 WP_0137        1
...                            ...
ORD_0479 SKU_0099 WP_0120        1
         SKU_0176 WP_0011        1
         SKU_0274 WP_0120        1
         SKU_0329 WP_0148        1
ORD_0480 SKU_0275 WP_0088        1

[1421 rows x 1 columns]

In [3]:
import numpy as np

def estimate_row_count_from_od_matrix(od_matrix):
    """
    OD Matrix에서 랙 간 거리 분포를 보고 한 줄에 몇 개 있는지, 전체 줄 수를 추정
    """
    row_len_candidates = []
    for rack in od_matrix.index :  # 일부만 샘플로 추정
        distances = od_matrix.loc[rack].sort_values()
        diffs = distances.diff().fillna(0)

        # 급격히 증가하는 지점 찾기 (거리 급증)
        jump_indices = np.where(diffs > 5)[0]  # 거리 단위 기준 조정 가능

        if len(jump_indices) > 0:
            row_len_candidates.append(jump_indices[0])  # 첫 번째 점프 위치

    # 최빈값 = 추정된 한 줄 랙 수
    from statistics import mode
    est_rack_count = mode(row_len_candidates) # 한 구역에 포함된 랙의 갯수

    # 전체 랙 수에서 줄 수 계산
    total_racks = len(od_matrix)
    est_row_count = total_racks // est_rack_count # 구역 갯수

    return est_rack_count

od_matrix = pd.read_csv('./data/Sample_OD_Matrix.csv', index_col= 0)
od_matrix.head()

res = estimate_row_count_from_od_matrix(od_matrix)
print(res)


14


In [4]:
import re

def extract_loc_number(loc_name):
    """
    LOC 이름(WP_0001 등)에서 숫자 부분만 추출하여 정수로 반환
    """
    match = re.search(r'\d+', loc_name)
    return int(match.group()) if match else None

def assign_zone_by_locnum(slap_df, row_len):
    """
    LOC_NUM을 기반으로 ZONE을 부여한다.
    
    Parameters:
    - slap_df: SLAP_LOC DataFrame
    - row_len: 추정된 한 줄에 있는 랙 수
    
    Returns:
    - ZONE이 부여된 DataFrame
    """
    slap_df = slap_df.copy()
    
    # LOC에서 숫자 추출
    slap_df['LOC_NUM'] = slap_df['LOC'].map(extract_loc_number)
    
    # ZONE 부여
    slap_df['ZONE'] = slap_df['LOC_NUM'].apply(
        lambda x: f"ZONE_{(x - 1) // row_len + 1}" if pd.notnull(x) else None
    )
    
    return slap_df
  
  

In [5]:
zone_assign = assign_zone_by_locnum(df, res)
zone_assign

,ORD_NO,SKU_CD,NUM_PCS,LOC,CART_NO,SEQ,LOC_NUM,ZONE
0,ORD_0330,SKU_0242,1,WP_0001,0,1,1,ZONE_1
1,ORD_0056,SKU_0274,1,WP_0120,0,2,120,ZONE_9
2,ORD_0138,SKU_0287,1,WP_0155,0,3,155,ZONE_12
3,ORD_0088,SKU_0046,1,WP_0156,0,4,156,ZONE_12
4,ORD_0092,SKU_0022,1,WP_0156,0,5,156,ZONE_12
...,...,...,...,...,...,...,...,...
1421,ORD_0167,SKU_0007,1,WP_0034,119,1,34,ZONE_3
1422,ORD_0167,SKU_0254,1,WP_0050,119,2,50,ZONE_4
1423,ORD_0167,SKU_0139,1,WP_0050,119,3,50,ZONE_4
1424,ORD_0167,SKU_0293,1,WP_0063,119,4,63,ZONE_5


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from math import ceil

# -----------------------------
# 2. 주문-SKU 매트릭스 & 주문-ZONE 매트릭스
# -----------------------------
order_sku_matrix = zone_assign.pivot_table(index='ORD_NO', columns='SKU_CD', aggfunc='size', fill_value=0)
order_zone_matrix = zone_assign.pivot_table(index='ORD_NO', columns='ZONE', aggfunc='size', fill_value=0)
order_zone_matrix = order_zone_matrix[
    sorted(order_zone_matrix.columns, key=lambda x: int(x.split('_')[-1]))
]
# -----------------------------
# 3. 유사도 계산
# -----------------------------
sku_sim = cosine_similarity(order_sku_matrix)
zone_sim = cosine_similarity(order_zone_matrix)

order_sku_matrix # ord / sku
order_zone_matrix # ord / zone

# 결합 유사도 (단순 평균)
combined_sim = (sku_sim + zone_sim) / 2
combined_dist = 1 - combined_sim  # KMeans는 거리 기반

# -----------------------------
# 4. KMeans 클러스터링 (4개씩 주문 묶기)
# -----------------------------
n_orders = order_sku_matrix.shape[0]
cart_capa = 4
n_clusters = ceil(n_orders / cart_capa)

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(combined_dist)

from collections import defaultdict

# 1. 클러스터 결과 저장
cluster_to_orders = defaultdict(list)
for ord_no, label in zip(order_sku_matrix.index, labels):
    cluster_to_orders[label].append(ord_no)

# 2. 각 클러스터에서 주문을 모아서 큰 리스트로 통합
#    유사한 순서를 보장하면서 카트당 4개씩 잘라 배정
all_orders = []
for cluster_orders in cluster_to_orders.values():
    all_orders.extend(cluster_orders)  # 군집 간 순서를 무시해도 됨

# 3. 4개씩 묶어서 CART_NO 부여
order_to_cart = {}
cart_id = 1
for i in range(0, len(all_orders), cart_capa):  # cart_capa = 4
    batch = all_orders[i:i+cart_capa]
    for ord_no in batch:
        order_to_cart[ord_no] = cart_id
    cart_id += 1

# 4. 최종 반영
zone_assign['CART_NO'] = zone_assign['ORD_NO'].map(order_to_cart)
# zone_assign.head(20)

c:\Users\chjin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [7]:
order_to_cart

{'ORD_0001': 1,
 'ORD_0046': 1,
 'ORD_0199': 1,
 'ORD_0227': 1,
 'ORD_0315': 2,
 'ORD_0335': 2,
 'ORD_0477': 2,
 'ORD_0002': 2,
 'ORD_0041': 3,
 'ORD_0264': 3,
 'ORD_0436': 3,
 'ORD_0454': 3,
 'ORD_0003': 4,
 'ORD_0010': 4,
 'ORD_0045': 4,
 'ORD_0099': 4,
 'ORD_0208': 5,
 'ORD_0245': 5,
 'ORD_0298': 5,
 'ORD_0394': 5,
 'ORD_0420': 6,
 'ORD_0441': 6,
 'ORD_0004': 6,
 'ORD_0258': 6,
 'ORD_0005': 7,
 'ORD_0054': 7,
 'ORD_0418': 7,
 'ORD_0467': 7,
 'ORD_0006': 8,
 'ORD_0083': 8,
 'ORD_0218': 8,
 'ORD_0286': 8,
 'ORD_0294': 9,
 'ORD_0322': 9,
 'ORD_0325': 9,
 'ORD_0353': 9,
 'ORD_0453': 10,
 'ORD_0480': 10,
 'ORD_0007': 10,
 'ORD_0163': 10,
 'ORD_0243': 11,
 'ORD_0349': 11,
 'ORD_0008': 11,
 'ORD_0076': 11,
 'ORD_0086': 12,
 'ORD_0095': 12,
 'ORD_0189': 12,
 'ORD_0009': 12,
 'ORD_0106': 13,
 'ORD_0107': 13,
 'ORD_0186': 13,
 'ORD_0191': 13,
 'ORD_0261': 14,
 'ORD_0283': 14,
 'ORD_0295': 14,
 'ORD_0340': 14,
 'ORD_0380': 15,
 'ORD_0390': 15,
 'ORD_0407': 15,
 'ORD_0450': 15,
 'ORD_0472': 16,

In [8]:
zone_assign = zone_assign.sort_values(['CART_NO',  'LOC'])
zone_assign.head(20)

# res = zone_assign.groupby('CART_NO').cumcount() + 1 # 1426

# zone_assign['SEQ'] = 

,ORD_NO,SKU_CD,NUM_PCS,LOC,CART_NO,SEQ,LOC_NUM,ZONE
579,ORD_0199,SKU_0075,1,WP_0043,1,1,43,ZONE_4
580,ORD_0199,SKU_0069,1,WP_0043,1,2,43,ZONE_4
581,ORD_0001,SKU_0246,1,WP_0049,1,3,49,ZONE_4
582,ORD_0199,SKU_0246,1,WP_0049,1,4,49,ZONE_4
583,ORD_0227,SKU_0254,1,WP_0050,1,5,50,ZONE_4
587,ORD_0046,SKU_0016,1,WP_0053,1,9,53,ZONE_4
588,ORD_0046,SKU_0251,1,WP_0053,1,10,53,ZONE_4
590,ORD_0046,SKU_0293,1,WP_0063,1,12,63,ZONE_5
591,ORD_0001,SKU_0267,1,WP_0101,1,13,101,ZONE_8
592,ORD_0046,SKU_0018,1,WP_0102,1,14,102,ZONE_8
